#### Step 1: Fetch Data From Binance

In [1]:
# import libraries
import requests
import pandas as pd

In [2]:
# Fetch the data
def fetch_binance(symbol="BTCUSDT", interval="1d", limit=1000):
    url = "https://api.binance.com/api/v3/klines"
    params = {"symbol": symbol, "interval": interval, "limit": limit}
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data)
    df.columns = ["open_time","open","high","low","close","volume",
                  "close_time","quote_asset_volume","num_trades",
                  "taker_base_volume","taker_quote_volume","ignore"]
    return df
df = fetch_binance()
print(df.head())

       open_time            open            high             low  \
0  1678233600000  22198.56000000  22287.00000000  21580.00000000   
1  1678320000000  21704.37000000  21834.99000000  20042.72000000   
2  1678406400000  20362.21000000  20367.78000000  19549.09000000   
3  1678492800000  20150.69000000  20686.51000000  19765.03000000   
4  1678579200000  20455.73000000  22150.00000000  20270.60000000   

            close           volume     close_time    quote_asset_volume  \
0  21705.44000000  301460.57272000  1678319999999   6641171704.24242130   
1  20362.22000000  443658.28584000  1678406399999   9403028764.70777380   
2  20150.69000000  618456.46710000  1678492799999  12344992788.91249570   
3  20455.73000000  427831.82133000  1678579199999   8651590672.46852580   
4  21997.11000000  430944.94288000  1678665599999   8982418413.80113910   

   num_trades taker_base_volume   taker_quote_volume ignore  
0     8018963   150008.02488000  3304860256.10499370      0  
1    10076702   

In [3]:
# Save the raw data in excel
df.to_csv(r"D:\DATA SCIENCE\SCHOOL PROJECTS\MACHINE LEARNING\data\raw\BTCUSDT_1d.csv", index=False)


#### Step 2: Data Cleaning & Basic Processing

In [4]:
# Convert time stamps to datetime
df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')


In [5]:
# Convert numeric columns to float
cols = ['open','high','low','close','volume','quote_asset_volume','taker_base_volume','taker_quote_volume']
df[cols] = df[cols].astype(float)


In [6]:
# Drop unused column
df = df.drop("ignore",axis=1)

In [7]:
# Sort by time , chronological order
df = df.sort_values('open_time').reset_index(drop=True)


In [8]:
# check for NaN
df.isnull().sum()


open_time             0
open                  0
high                  0
low                   0
close                 0
volume                0
close_time            0
quote_asset_volume    0
num_trades            0
taker_base_volume     0
taker_quote_volume    0
dtype: int64

In [9]:
# check for duplicate rows
duplicate_rows = df[df.duplicated()]
duplicate_rows


,open_time,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,taker_base_volume,taker_quote_volume


##### Returns

In [10]:
# Returns
# Daily & weekly returns - [Capture Momentum] return of a financial security during regular trading hours(1 day and 7 days)
df['close_pct_1d'] = df['close'].pct_change(1)
df['close_pct_7d'] = df['close'].pct_change(7)

# Rolling volatility - [Capture Market Activity/ Risk] Calculates the volatility of an asset's price movements over a specified period.(It measures the degree of variation in the price series over time)
df['vol_7d'] = df['close_pct_1d'].rolling(7).std()
df['vol_30d'] = df['close_pct_1d'].rolling(30).std()


In [11]:
# Save the processed excel
df.to_csv(r"D:\DATA SCIENCE\SCHOOL PROJECTS\MACHINE LEARNING\data\processed/BTCUSDT_1d_processed.csv", index=False)


In [12]:
df.head()

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,taker_base_volume,taker_quote_volume,close_pct_1d,close_pct_7d,vol_7d,vol_30d
0,2023-03-08,22198.56,22287.00,21580.00,21705.44,301460.57272,2023-03-08 23:59:59.999,6.641172e+09,8018963,150008.02488,3.304860e+09,NaN,NaN,NaN,NaN
1,2023-03-09,21704.37,21834.99,20042.72,20362.22,443658.28584,2023-03-09 23:59:59.999,9.403029e+09,10076702,217900.59621,4.619804e+09,-0.061884,NaN,NaN,NaN
2,2023-03-10,20362.21,20367.78,19549.09,20150.69,618456.46710,2023-03-10 23:59:59.999,1.234499e+10,12106261,308155.80993,6.151475e+09,-0.010388,NaN,NaN,NaN
3,2023-03-11,20150.69,20686.51,19765.03,20455.73,427831.82133,2023-03-11 23:59:59.999,8.651591e+09,10412300,213151.47855,4.310534e+09,0.015138,NaN,NaN,NaN
4,2023-03-12,20455.73,22150.00,20270.60,21997.11,430944.94288,2023-03-12 23:59:59.999,8.982418e+09,9721933,218639.06880,4.558128e+09,0.075352,NaN,NaN,NaN


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   open_time           1000 non-null   datetime64[ns]
 1   open                1000 non-null   float64       
 2   high                1000 non-null   float64       
 3   low                 1000 non-null   float64       
 4   close               1000 non-null   float64       
 5   volume              1000 non-null   float64       
 6   close_time          1000 non-null   datetime64[ns]
 7   quote_asset_volume  1000 non-null   float64       
 8   num_trades          1000 non-null   int64         
 9   taker_base_volume   1000 non-null   float64       
 10  taker_quote_volume  1000 non-null   float64       
 11  close_pct_1d        999 non-null    float64       
 12  close_pct_7d        993 non-null    float64       
 13  vol_7d              993 non-null    float64      

In [14]:
df.describe()

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,taker_base_volume,taker_quote_volume,close_pct_1d,close_pct_7d,vol_7d,vol_30d
count,1000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000,1.000000e+03,1.000000e+03,1000.000000,1.000000e+03,999.000000,993.000000,993.000000,970.000000
mean,2024-07-19 12:00:00,67596.167810,68798.062880,66361.112640,67660.697540,38506.202862,2024-07-20 11:59:59.999000320,2.091303e+09,2.697535e+06,18979.878283,1.027550e+09,0.001684,0.012048,0.022066,0.023158
min,2023-03-08 00:00:00,20150.690000,20367.780000,19549.090000,20150.690000,3282.173520,2023-03-08 23:59:59.999000,2.508740e+08,3.108520e+05,1514.419590,1.165812e+08,-0.085387,-0.191150,0.003164,0.008730
25%,2023-11-12 18:00:00,36388.075000,37361.750000,35607.520000,36541.807500,17559.174262,2023-11-13 17:59:59.999000064,1.053586e+09,1.152395e+06,8471.924338,5.100617e+08,-0.010627,-0.026885,0.014234,0.018439
50%,2024-07-19 12:00:00,65327.330000,66700.815000,63813.635000,65365.010000,27578.558780,2024-07-20 11:59:59.999000064,1.669912e+09,1.870772e+06,13261.277570,8.155861e+08,0.000428,0.004956,0.020573,0.022620
75%,2025-03-26 06:00:00,96691.782500,98288.750000,95208.090000,96691.782500,41087.563232,2025-03-27 05:59:59.999000064,2.485502e+09,3.445931e+06,20225.699702,1.222804e+09,0.012818,0.042239,0.027188,0.026741
max,2025-12-01 00:00:00,124658.540000,126199.630000,123084.000000,124658.540000,699360.934230,2025-12-01 23:59:59.999000,1.746531e+10,1.522359e+07,351739.119940,8.783916e+09,0.118834,0.359513,0.065737,0.044142
std,NaN,31035.484466,31464.300639,30518.228063,31008.072366,59947.112194,NaN,1.807634e+09,2.335037e+06,30125.347478,9.064653e+08,0.024479,0.065261,0.010344,0.006564
